### DataFrame과 Dataset
- 결과를 생성하기 위해 어떤 데이터에 어떤 연산을 적용해야하는지 정의하는 지연 연산의 **실행 계획**
- 불변성을 가짐
- 액션을 호출하면 스파크는 실제로 transformation을 실행하고 결과를 반환함
- 스키마: DataFrame의 컬럼명과 데이터 타입을 정의함. 스파크는 자체 데이터 타입을 갖고 있음

- Dataset은 scala와 java에서만 지원되며, Dataset의 경우 컴파일 타임에 스키마에 명시된 데이터 타입의 일치여부를 검증함
- DataFrame은 런타임이 되어서야 데이터 타입의 일치 여부를 검증함
- DataFrame은 Row 타입으로 구성된 Dataset임. 이 형식은 "연산에 최적화된 인메모리 포맷"의 내부적 표현 방식임

### 컬럼과 로우
- 컬럼: 테이블의 컬럼과 동일. 단순 데이터 타입, 배열과 같은 복합 데이터 타입, null을 표현함
- 로우: 데이터 레코드. 데이터소스에서 얻거나 직접 생성 가능

In [9]:
// 스파크의 덧셈 연산
val df = spark.range(500).toDF("number") //0~499까지의 값이 할당된 500 row의 df 생성
df.select(df.col("number")+10).take(10)  //각 로우에 10을 더함

df: org.apache.spark.sql.DataFrame = [number: bigint]
res8: Array[org.apache.spark.sql.Row] = Array([10], [11], [12], [13], [14], [15], [16], [17], [18], [19])


In [12]:
// scala에서 스파크 데이터 타입 사용
import org.apache.spark.sql.types._

val b = ByteType

import org.apache.spark.sql.types._
b: org.apache.spark.sql.types.ByteType.type = ByteType


### 구조적 API의 실행 과정
- DF/DS/SQL을 이용해 코드 작성 &rarr; 논리적 실행 계획 생성 &rarr; 최적화 및 물리적 실행 계획으로 변환 &rarr; 연산 실행
- 논리적 실행 계획: catalyst optimizer가 실행 계획을 최적화
- 물리적 실행 계획: 논리적 실행 계획을 클러스터 환경에서 실행하는 방법 정의. 최종적으로 RDD transformation으로 변환되어 실행

### 구조적 API의 기본 기능
- 스키마: DataFrame의 컬럼명과 데이터 타입을 정의. 스키마를 데이터에서 읽어오거나 스스로 만들 수도 있음
- 사용자는 표현식을 사용하여 컬럼의 선택, 조작, 제거를 수행함. 컬럼은 사실상 표현식으로 레코드의 값을 단순하게 나타내는 논리적 구조임
- 표현식: DataFrame 레코드의 여러 값에 대한 transformation 집합
- row: 스파크는 레코드를 Row 객체로 표현함. Row 객체는 스키마 정보를 갖고 있지 않음

In [1]:
// DataFrame 생성
val df = spark.read.format("json")
  .load("c:/SparkDG/data/flight-data/json/2015-summary.json")

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-VMF5KGF:4040
SparkContext available as 'sc' (version = 2.3.2, master = local[*], app id = local-1597022541944)
SparkSession available as 'spark'


df: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [2]:
//스키마 출력
df.printSchema

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [16]:
//column 생성: col 함수 사용
import org.apache.spark.sql.functions.{col, column, expr}
col("someColumnName")
expr("someColumnName2")  // expr 함수에 인수로 컬럼 표현식을 넣으면 col 함수와 동일하게 사용 가능

import org.apache.spark.sql.functions.{col, column, expr}
res14: org.apache.spark.sql.Column = someColumnName2


In [4]:
//row 생성 및 데이터 접근. row 생성시 df와 같은 순서로 값을 명시해야 함
import org.apache.spark.sql.Row
val myRow = Row("Hello", null, 1, false)

myRow.getString(0)
myRow.getInt(2)

import org.apache.spark.sql.Row
myRow: org.apache.spark.sql.Row = [Hello,null,1,false]
res2: Int = 1


### select, selectExpr 메서드
- DataFrame에서 SQL 사용 가능하게 함

In [5]:
// 칼럼명을 문자열로 참조
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [7]:
// expr 함수 사용 (문자열이 아닌 표현식)
df.select(expr("DEST_COUNTRY_NAME as destination")).show(2)

+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



In [9]:
// selectExpr 메서드: 복잡한 표현식을 간단하게 표현
df.selectExpr(
    "*", //모든 컬럼 선택
    "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry")
  .show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [10]:
//컬럼 추가 (칼럼명, 값)
import org.apache.spark.sql.functions.lit
df.withColumn("numberOne", lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



import org.apache.spark.sql.functions.lit


In [11]:
//컬럼명 변경
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").columns

res9: Array[String] = Array(dest, ORIGIN_COUNTRY_NAME, count)


In [12]:
//row 필터링 (where, filter)
df.filter(col("count") < 2).show(2)
df.where("count < 2").show(2)

// SELECT * FROM dfTable WHERE count < LIMIT 2

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



In [22]:
//row 정렬하기
df.sort("count").show(5)
df.orderBy(desc("DEST_COUNTRY_NAME"), asc("count")).show(2)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|           Zambia|      United States|    1|
|        Venezuela|      United States|  290|
+-----------------+-------------------+-----+
only showing top 2 rows



## 다양한 데이터 타입
- [예제] 소매 데이터

In [23]:
val df = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("c:/SparkDG/data/retail-data/by-day/2010-12-01.csv")
df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



df: org.apache.spark.sql.DataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]


### Boolean
- 문자열 표현식에 조건절 명시 가능

In [29]:
df.where("InvoiceNo <> 536365").show(5, false)

+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                  |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
|536366   |22633    |HAND WARMER UNION JACK       |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536366   |22632    |HAND WARMER RED POLKA DOT    |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536367   |84879    |ASSORTED COLOUR BIRD ORNAMENT|32      |2010-12-01 08:34:00|1.69     |13047.0   |United Kingdom|
|536367   |22745    |POPPY'S PLAYHOUSE BEDROOM    |6       |2010-12-01 08:34:00|2.1      |13047.0   |United Kingdom|
|536367   |22748    |POPPY'S PLAYHOUSE KITCHEN    |6       |2010-12-01 08:34:00|2.1      |13047.0   |United Kingdom|
+---------+---------+-----------------------------+--------+----

In [38]:
spark.sql("""
SELECT * FROM dfTable
Where StockCode in ("DOT")
AND (UnitPrice > 600 OR instr(Description, "POSTAGE") >=1)
""").show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [39]:
val priceFilter = col("UnitPrice") > 600                    //bool
val descripFilter = col("Description").contains("POSTAGE")  //bool
df.where(col("StockCode").isin("DOT")).where(priceFilter.or(descripFilter))
  .show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



priceFilter: org.apache.spark.sql.Column = (UnitPrice > 600)
descripFilter: org.apache.spark.sql.Column = contains(Description, POSTAGE)


### 수치형 데이터

In [40]:
// (수량 * 단위가격)^2 + 5 값 구하기 
val fabricatedQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerID"), fabricatedQuantity.alias("realQuantity")).show(2)

+----------+------------------+
|CustomerID|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



fabricatedQuantity: org.apache.spark.sql.Column = (POWER((Quantity * UnitPrice), 2.0) + 5)


In [43]:
df.selectExpr(
  "CustomerID", "(POWER(Quantity * UnitPrice, 2.0) + 5) as realQuantity"
).show(2)

+----------+------------------+
|CustomerID|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [44]:
// 상관관계 구하기
df.select(corr("Quantity", "UnitPrice")).show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+



### 날짜 및 타임스탬프

In [51]:
val dateDF = spark.range(10)
  .withColumn("today", current_date())
  .withColumn("now", current_timestamp())
dateDF.createOrReplaceTempView("dateTable")
dateDF.printSchema()

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)



dateDF: org.apache.spark.sql.DataFrame = [id: bigint, today: date ... 1 more field]


In [52]:
dateDF.select(date_sub(col("today"), 5), date_add(col("today"), 5)).show(1)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2020-08-05|        2020-08-15|
+------------------+------------------+
only showing top 1 row



In [55]:
//datediff 함수, months_between 함수
dateDF.select(
    to_date(lit("2016-01-01")).alias("start"),           //to_date: 문자열을 날짜로 변환 (형식에 맞지 않는 값일 경우 null)
    to_date(lit("2017-05-22")).alias("end"))
  .select(datediff(col("end"), col("start"))).show(1)
dateDF.select(
    to_date(lit("2016-01-01")).alias("start"),
    to_date(lit("2017-05-22")).alias("end"))
  .select(months_between(col("end"), col("start"))).show(1)

+--------------------+
|datediff(end, start)|
+--------------------+
|                 507|
+--------------------+
only showing top 1 row

+--------------------------+
|months_between(end, start)|
+--------------------------+
|               16.67741935|
+--------------------------+
only showing top 1 row



In [56]:
// timestamp의 경우 항상 날짜 포맷을 지정해야 함
val dateFormat = "yyyy-dd-MM"
spark.range(1).select(to_timestamp(lit("2017-12-11"), dateFormat)).show()

+----------------------------------------+
|to_timestamp('2017-12-11', 'yyyy-dd-MM')|
+----------------------------------------+
|                     2017-11-12 00:00:00|
+----------------------------------------+



dateFormat: String = yyyy-dd-MM


### null

In [58]:
// null 값을 다른 값으로 채워넣음
df.na.fill("none":String)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows

+--